In [1]:
import requests
import time
import pymysql
import os
from random import uniform
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def rest_get(apilink):
    print(apilink)
    resp = requests.get(apilink)
    if resp.status_code != 200:
        # This means something went wrong.
        raise Exception('GET {} {}'.format(apilink, resp.status_code))
    else:
        return resp

### API approach, has limits per day - not worth it, continue to full scraping approach

In [3]:
'''
links = ['https://github.com/poloclub/cnn-explainer']
github = 'https://api.github.com/'
gituser = 'https://github-user.now.sh/?username='
    
for l in links:
    repo_split = l.split('/')
    owner, repo = repo_split[-2], repo_split[-1]
    print('Requestiong: ',owner, repo)
    resp = rest_get(f'{github}repos/{owner}/{repo}/contributors')
    for user in resp.json():
        print(f'user "{user["login"]}" has {user["contributions"]} contributions')
        user_info = rest_get(gituser+user["login"])
        print(f"Name: {user_info['name']}, Description: {user_info['description']}")
'''
    

'\nlinks = [\'https://github.com/poloclub/cnn-explainer\']\ngithub = \'https://api.github.com/\'\ngituser = \'https://github-user.now.sh/?username=\'\n    \nfor l in links:\n    repo_split = l.split(\'/\')\n    owner, repo = repo_split[-2], repo_split[-1]\n    print(\'Requestiong: \',owner, repo)\n    resp = rest_get(f\'{github}repos/{owner}/{repo}/contributors\')\n    for user in resp.json():\n        print(f\'user "{user["login"]}" has {user["contributions"]} contributions\')\n        user_info = rest_get(gituser+user["login"])\n        print(f"Name: {user_info[\'name\']}, Description: {user_info[\'description\']}")\n'

### Saving author contributions to txt file

In [4]:
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
cursor=conn.cursor()

In [5]:

repos = []
cursor.execute("SELECT DISTINCT krw.repo.repo_url FROM krw.paper \
INNER JOIN krw.repo ON krw.repo.paper_url = krw.paper.paper_url \
WHERE krw.paper.makg_sameAs <> 'notfound'")
for r in cursor:
    repos.append(list(r)[0])
'''
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))
'''

'\ncursor.execute("SELECT krw.repo.paper_url, repo_url, author                FROM krw.repo INNER JOIN krw.paper_author                ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")\nfor r in cursor:\n    data.append(list(r))\n'

In [6]:
len(repos)

11266

In [7]:
filename = 'scraped_data.txt'
if not os.path.exists(filename):
    file = open(filename, 'w+')
    file.close()

In [8]:
# Remove repos we already saved into file
with open(filename) as fp:
    line = fp.readline().split()
    while line:
        current_repo = line[0]
        if current_repo in repos:
            repos.remove(current_repo)
        line = fp.readline().split()
len(repos)

8540

In [9]:
repos.index('https://github.com/facebook/fb.resnet.torch')

ValueError: 'https://github.com/facebook/fb.resnet.torch' is not in list

In [10]:
missed = []

In [18]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # Firefox()

for repo in repos[8001:8100]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            try:
                scrape = rest_get(f'https://github.com/{username}')
            except:
                continue
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

[WDM] - Looking for [chromedriver 81.0.4044.138 linux64] driver in cache 
[WDM] - File found in cache by path [/home/kanter/.wdm/drivers/chromedriver/81.0.4044.138/linux64/chromedriver]


https://github.com/ulises-jeremias/prototypical-networks-tf
NOT FOUND/LOADED

https://github.com/sannebh/riskaverse_rl
NOT FOUND/LOADED

https://github.com/DeepMotionAIResearch/DenseASPP
https://github.com/DeepMotionAIResearch
name: , bio: 
User DeepMotionAIResearch: commits 5 , changed lines + 14 - 10

https://github.com/Ipsedo/IterativeMachineTeaching
NOT FOUND/LOADED

https://github.com/saurabhkumar8112/DQNN_papers
NOT FOUND/LOADED

https://github.com/diegoschapira/CNN-Text-Classifier-using-Keras
NOT FOUND/LOADED

https://github.com/Tanasho0928/ncm
NOT FOUND/LOADED

https://github.com/gcwl/pytorch-dcgan
NOT FOUND/LOADED

https://github.com/franckalain/Tensorflow-Projects-ChatBot
NOT FOUND/LOADED

https://github.com/meijieru/yet_another_mobilenet_series
https://github.com/JiahuiYu
name: , bio: Research Scientist, Google Brain
User JiahuiYu: commits 7 , changed lines + 2,733 - 142
https://github.com/meijieru
name: Jieru Mei, bio: Ph.D. student in Computer Science
User meijieru: commit

NOT FOUND/LOADED

https://github.com/JunkyByte/HER_DQN
NOT FOUND/LOADED

https://github.com/DongyaoZhu/VQ-VAE-WaveNet
https://github.com/DongyaoZhu
name: dyz, bio: software engineer, deep learning enthusiast
User DongyaoZhu: commits 33 , changed lines + 322,000 - 197,510

https://github.com/Byte7/GANs
NOT FOUND/LOADED

https://github.com/Wakeupbuddy/amodalAPI
https://github.com/Wakeupbuddy
name: Yan Zhu, bio: 
User Wakeupbuddy: commits 14 , changed lines + 18,711 - 15,267

https://github.com/EugeneNdiaye/safe_grid_search
NOT FOUND/LOADED

https://github.com/waynezhanghk/gacluster
NOT FOUND/LOADED

https://github.com/floraxinru/NLP_HotelReviews
https://github.com/floraxinru
name: Xinru Cheng, bio: From physics research and teaching to Data Science, I'm excited to solve difficult problems in an Agile environment, to make a positive impact in the world.
User floraxinru: commits 26 , changed lines + 6,677 - 4,867

https://github.com/mandubian/neural-ode
NOT FOUND/LOADED

https://github.com

In [21]:
driver = webdriver.Firefox() # 

for repo in repos[5901:6000]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            try:
                scrape = rest_get(f'https://github.com/{username}')
            except:
                continue
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/JUrban/deepmath
NOT FOUND/LOADED

https://github.com/stratospark/food-101-keras
https://github.com/stratospark
name: Patrick Rodriguez, bio: 
User stratospark: commits 17 , changed lines + 9,663 - 1,889

https://github.com/alishdipani/Neural-Style-Transfer-Audio
https://github.com/alishdipani
name: Alish Dipani, bio: Research Intern at CNRS, France || Computer science Undergrad at BITS Goa || Intel Student Ambassador for Artificial Intelligence || GSoC 2019 with @SciRuby
User alishdipani: commits 19 , changed lines + 440 - 53

https://github.com/jcoreyes/ex2
NOT FOUND/LOADED

https://github.com/PaddlePaddle/XWorld
https://github.com/skylian
name: Xiaochen Lian, bio: 
User skylian: commits 20 , changed lines + 6,693 - 1,681
https://github.com/Haichao-Zhang
name: Haichao Zhang, bio: 
User Haichao-Zhang: commits 19 , changed lines + 1,577 - 320
https://github.com/wangkuiyi
name: Yi Wang, bio: AI system engineer
User wangkuiyi: commits 1 , changed lines + 1 - 0
https://g

NOT FOUND/LOADED

https://github.com/vbookshelf/Skin-Lesion-Analyzer
NOT FOUND/LOADED

https://github.com/sharathsarangmath/Vector-Capsule-Network-for-wild-animal-species-recognition-in-Camera-trap-images.
https://github.com/sharathsarangmath
name: Sharath Sarangmath, bio: 
User sharathsarangmath: commits 55 , changed lines + 1,997 - 967

https://github.com/abramjos/Scene-boundary-detection
https://github.com/abramjos
name: Abraham Jose, bio: Computer Vision Engineer. Graduate student at UCF's Computer Science department focusing on Computer Vision, especially in temporal analysis of images.
User abramjos: commits 7 , changed lines + 69 - 18

https://github.com/trakaros/MPIIGaze
https://github.com/caxelos
name: Christos Axelos, bio: University Of Thessaly, Greece
User caxelos: commits 284 , changed lines + 393,432 - 8,834
https://github.com/oaxelou
name: Olympia Axelou, bio: University of Thessaly, Greece, department of Electrical and Computer Enginnering
User oaxelou: commits 1 , chan

name: Ibrahim Sharaf ElDen, bio: NLP Research Engineer
User ibrahimsharaf: commits 25 , changed lines + 25,526 - 211
https://github.com/choas
name: Lars Gregori, bio: 
User choas: commits 1 , changed lines + 2 - 2
https://github.com/avinashsai
name: Avinash Madasu, bio: 
User avinashsai: commits 1 , changed lines + 13 - 5
https://github.com/Ayatallah
name: Ayatallah Amr, bio: 
User Ayatallah: commits 1 , changed lines + 25,252 - 25,109

https://github.com/ZhimingZhou/AM-GAN
NOT FOUND/LOADED

https://github.com/mhsung/deep-functional-dictionaries
NOT FOUND/LOADED

https://github.com/daigo0927/PWC-Net_tf
https://github.com/daigo0927
name: Daigo HIROOKA, bio: Machine learning engineer.
Interested in image/video processing, representation learning in deep neural network and practical installation of machine learning
User daigo0927: commits 8 , changed lines + 179 - 13

https://github.com/stegua/dotlib
NOT FOUND/LOADED

https://github.com/AIBluefisher/ResearchWork
NOT FOUND/LOADED

https://

name: Prerit Jaiswal, bio: AI Research Engineer | PhD -
 Theoretical Physics | Self-driving cars enthusiast
User preritj: commits 49 , changed lines + 151,949 - 14,715
https://github.com/liufeng2012
name: , bio: 
User liufeng2012: commits 22 , changed lines + 907 - 691
https://github.com/aaronfriedman6
name: Aaron Friedman, bio: 
User aaronfriedman6: commits 20 , changed lines + 291,092 - 290,140
https://github.com/LanceJingChen
name: , bio: 
User LanceJingChen: commits 19 , changed lines + 15,112 - 4,023
https://github.com/s-nepal
name: , bio: 
User s-nepal: commits 15 , changed lines + 82,162 - 133
https://github.com/lowspin
name: Tze-Ping Low, bio: Learn, Make, Connect. Repeat.
User lowspin: commits 8 , changed lines + 27,422 - 16,119
https://github.com/mandarup
name: Mandar Upadhye, bio: 
User mandarup: commits 6 , changed lines + 1,690 - 1,671
https://github.com/randxie
name: Rand Xie, bio: Programmer, Kaggler, Heavy Pytorch User; Working on scalable ML serving infrastructure.
Use

In [20]:
len(missed)

286

### Getting list of repositories and authors from the file

In [10]:
data = []
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 10000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))

In [11]:
author_dict = {}
# Version with name as one string
for d in data:
    if d[1] not in author_dict:
        author_dict[d[1]]=[d[2]]
    else:
        author_dict[d[1]].append(d[2])
print(len(author_dict.keys()))

2079


In [12]:
author_dict['https://github.com/google/tf_mesh_renderer']

['Kyle Genova',
 'Forrester Cole',
 'Aaron Maschinot',
 'Aaron Sarna',
 'Daniel Vlasic',
 'William T. Freeman']

In [13]:
filename_mini = 'scraped_data_mini.txt' # use full version later

In [14]:
paper_data = {}

with open(filename) as fp:
    line = fp.readline().split()
    current_repo = line[0]
    contributors = [line[1:]]
    while line:
        line = fp.readline().split()
        if len(line) and line[0]==current_repo:
            contributors.append(line[1:])
        else:
            print(f'Doing {current_repo} with {len(contributors)} contributors')
            print(f'Authors: {author_dict[current_repo]}')
            paper_data[current_repo] = {}
            for cont in contributors:
                author_name = cont[1].replace("_"," ")
                found_name = author_name in author_dict[current_repo]
                print(f"{author_name} in {found_name}")
                if found_name:
                    paper_data[current_repo][author_name] = cont[2:]
            # after saving previous authors, load new repo
            current_repo = line[0]
            contributors = [line[1:]]                    

Doing https://github.com/google/tf_mesh_renderer with 3 contributors
Authors: ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
NULL in False
Kyle Genova in True
NULL in False
Doing https://github.com/twke18/Adaptive_Affinity_Fields with 3 contributors
Authors: ['Tsung-Wei Ke', 'Jyh-Jing Hwang', 'Ziwei Liu', 'Stella X. Yu']
Tsung-Wei Ke in True
Jyh-Jing Hwang in True
Tsung-Wei Ke in True
Doing https://github.com/VladyslavUsenko/basalt-mirror with 3 contributors
Authors: ['Vladyslav Usenko', 'Nikolaus Demmel', 'Daniel Cremers', 'David Schubert', 'Thore Goll', 'Nikolaus Demmel', 'Vladyslav Usenko', 'Jörg Stückler', 'Daniel Cremers', 'Vladyslav Usenko', 'Nikolaus Demmel', 'David Schubert', 'Jörg Stückler', 'Daniel Cremers', 'Christiane Sommer', 'Vladyslav Usenko', 'David Schubert', 'Nikolaus Demmel', 'Daniel Cremers']
Vladyslav Usenko in True
Nikolaus Demmel in True
NULL in False
Doing https://github.com/new-okaerinasai/bwgan_pytorc

KeyError: 'https://github.com/benedekrozemberczki/karateclub'

In [15]:
len(paper_data)

212

### Creating graph

In [16]:
from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF

In [ ]:
g = Graph()


### Full scraping approach

In [61]:
paper_data = {}
driver = webdriver.Firefox()

for paper in final_data:
    repo = paper[1]
    print(repo, paper[2]) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(6) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes[0:10]:   # take just top 10 contributors, some repos might have 100+ contributors
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name} and {type(name)}, bio: {bio}')
        
        if name!="" and name in paper[2]:
            if paper[0] not in paper_data:
                paper_data[paper[0]]= [paper[1], {}]
            #print(info_line[1])
            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            paper_data[paper[0]][1][name] = [commits, added, deleted]
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/nianticlabs/monodepth2 ['Oisin Mac Aodha', 'Michael Firman', 'Gabriel J. Brostow']
https://github.com/mdfirman
name:  and <class 'str'>, bio: 
https://github.com/mrharicot
name: Clément Godard and <class 'str'>, bio: 
https://github.com/daniyar-niantic
name: Daniyar Turmukhambetov and <class 'str'>, bio: Senior Research Scientist at Niantic

https://github.com/google/tf_mesh_renderer ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
https://github.com/fcole
name:  and <class 'str'>, bio: 
https://github.com/kylegenova
name: Kyle Genova and <class 'str'>, bio: 
User kylegenova: commits 2 , changed lines + 18 - 1
https://github.com/TerisGH
name:  and <class 'str'>, bio: 

https://github.com/Panda-Peter/visda2019-multisource ['Yingwei Pan', 'Yehao Li', 'Qi Cai', 'Yang Chen', 'Ting Yao']
https://github.com/Panda-Peter
name:  and <class 'str'>, bio: Yingwei Pan

https://github.com/yanggeng1995/GAN-TTS ['Mikołaj Bińk


https://github.com/DylanWusee/pointconv_pytorch ['Charles R. Qi', 'Hao Su', 'Kaichun Mo', 'Leonidas J. Guibas']


WebDriverException: Message: Failed to decode response from marionette


In [47]:
paper_data

{'https://paperswithcode.com/paper/unsupervised-training-for-3d-morphable-model': ['https://github.com/google/tf_mesh_renderer',
  {'Kyle Genova': ['2', '18', '1']}],
 'https://paperswithcode.com/paper/adaptive-affinity-fields-for-semantic': ['https://github.com/twke18/Adaptive_Affinity_Fields',
  {'Tsung-Wei Ke': ['2', '24', '6'], 'Jyh-Jing Hwang': ['9', '31', '26']}],
 'https://paperswithcode.com/paper/improving-information-retrieval-results-for': ['https://github.com/adelra/query-expansion',
  {'Adel Rahimi': ['2', '99,652', '0']}],
 'https://paperswithcode.com/paper/the-double-sphere-camera-model': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'https://paperswithcode.com/paper/the-tum-vi-benchmark-for-evaluating-visual': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'ht

In [3]:
repos = ['https://github.com/poloclub/cnn-explainer', 'https://github.com/openai/jukebox']

In [4]:
driver = webdriver.Firefox()

for repo in repos:
    print(repo)
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(2)
    #delay of some kind wait for load time.sleep(3) or selenium wait for an element to be visible
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes:
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        #print(info_line[1])
        commits = info_line[1].find('a').text.strip().split()[0]
        added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
        deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
        print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name}, bio: {bio}')
        print()

https://github.com/poloclub/cnn-explainer
User xiaohk: commits 189 , changed lines + 35,562 - 14,943
https://github.com/xiaohk
name: Jay Wang, bio: Machine Learning PhD student at Georgia Tech.


Life is not always 🦄 and 🌈, but my favorite emoji is 😉

User rturko3: commits 58 , changed lines + 2,870 - 950
https://github.com/rturko3
name: , bio: CS Undergraduate @ Georgia Tech

User oshaikh13: commits 8 , changed lines + 117 - 74
https://github.com/oshaikh13
name: Omar Shaikh, bio: no u

https://github.com/openai/jukebox
User prafullasd: commits 49 , changed lines + 54,045 - 301
https://github.com/prafullasd
name: Prafulla Dhariwal, bio: 

User heewooj: commits 24 , changed lines + 393 - 344
https://github.com/heewooj
name: Heewoo Jun, bio: 

User jongwook: commits 5 , changed lines + 18 - 13
https://github.com/jongwook
name: Jong Wook Kim, bio: 

User mcleavey: commits 4 , changed lines + 964 - 3
https://github.com/mcleavey
name: Christine Payne, bio: OpenAI Researcher. Stanford Master